In [12]:
# Orbital Materials - Pretrained models for atomic simulations
# https://github.com/orbital-materials/orb-models/blob/main/README.md

In [13]:
# Direct Usage
import ase
from ase.build import bulk
from orb_models.forcefield import pretrained
from orb_models.forcefield import atomic_system
from orb_models.forcefield.base import batch_graphs

orbff = pretrained.orb_v1()
atoms = bulk('Fe', 'fcc', a=2.457, cubic=True)
graph = atomic_system.ase_atoms_to_atom_graphs(atoms)

# Optionally, batch graphs for faster inference
# graph = batch_graphs([graph, graph, ...])

result = orbff.predict(graph)
print(result)

# Convert to ASE atoms (this will also unbatch the results)
atoms = atomic_system.atom_graphs_to_ase_atoms(
    graph,
    energy=result["graph_pred"],
    forces=result["node_pred"],
    stress=result["stress_pred"]
)

print("\n", atoms)


{'graph_pred': tensor([-7.2206], grad_fn=<AddBackward0>), 'stress_pred': tensor([[-2.5782e+00, -2.5732e+00, -2.5759e+00,  5.4228e-05, -2.1484e-04,
         -2.3494e-05]], grad_fn=<AddBackward0>), 'node_pred': tensor([[0.6993, 0.1705, 0.3907],
        [0.6916, 0.1563, 0.4355],
        [0.5888, 0.1433, 0.4306],
        [0.5942, 0.1610, 0.4040]], grad_fn=<AddBackward0>)}

 [Atoms(symbols='Fe4', pbc=True, cell=[2.4570000171661377, 2.4570000171661377, 2.4570000171661377], tags=..., calculator=SinglePointCalculator(...))]


In [14]:
# Using ASE Calculator
import ase
from ase.build import bulk
from orb_models.forcefield import pretrained
from orb_models.forcefield.calculator import ORBCalculator


orbff = pretrained.orb_v1() # Or choose another model using ORB_PRETRAINED_MODELS[model_name]()
calc = ORBCalculator(orbff, device="cpu")  # or device="cuda"
atoms = bulk('Cu', 'fcc', a=3.58, cubic=True)

print(atoms.set_calculator(calc))
print(atoms.get_potential_energy())

None
-16.34379768371582


In [15]:
# Using the calculator with ASE compatible calculator
from ase.optimize import BFGS

# Rattle the atoms to get them out of the minimum energy configuration
atoms.rattle(0.5)
print("Rattled Energy:", atoms.get_potential_energy())

calc = ORBCalculator(orbff, device="cpu")
dyn = BFGS(atoms)
dyn.run(fmax=0.01)
print("Optimized Energy:", atoms.get_potential_energy())

Rattled Energy: -11.943151473999023
      Step     Time          Energy         fmax
BFGS:    0 15:54:36      -11.943151       12.1699
BFGS:    1 15:54:36      -14.303596        3.9488
BFGS:    2 15:54:36      -15.291109        2.8918
BFGS:    3 15:54:36      -16.141619        1.2014
BFGS:    4 15:54:36      -16.226456        1.2086
BFGS:    5 15:54:36      -16.276001        1.0189
BFGS:    6 15:54:37      -16.322590        0.5441
BFGS:    7 15:54:37      -16.338596        0.2140
BFGS:    8 15:54:37      -16.342379        0.1471
BFGS:    9 15:54:37      -16.344131        0.1399
BFGS:   10 15:54:37      -16.346653        0.0976
BFGS:   11 15:54:37      -16.346901        0.0394
BFGS:   12 15:54:37      -16.345810        0.0111
BFGS:   13 15:54:37      -16.345680        0.0074
Optimized Energy: -16.345680236816406
